In [8]:
#!/usr/bin/env python3
import os
from pathlib import Path
import numpy as np
import pandas as pd

from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas

"""
===============================================================================
DAILY STATEMENT PDF GENERATOR (LOT-BASED POSITIONS + STOP-RISK, FUTURES BACKTEST)
===============================================================================

One page per date, showing:
1) Account equity
2) Margin utilization % and $
3) Total notional exposure (gross) and notional/equity
4) Total risk to stops ($ and % of equity)
5) New transactions broken out by BTO / STO / BTC / STC
6) Return on closed trades (realized P&L and % of prior-day equity)
7) Existing portfolio lots (NOT averaged):
   - entry_date, entry_price, stop_loss_price, qty (contracts), multiplier,
     notional, margin_per_contract, margin_used, unrealized_pnl,
     risk_to_stop ($) and risk_to_stop (% equity)

KEY RULE (per your request):
- Every OPEN transaction (BTO/STO) creates a DISTINCT LOT.
- Lots are NOT averaged together across time.
- Close transactions (BTC/STC) close lots FIFO by default (configurable).

Risk to stop definition (per lot):
- LONG:  max(0, close - stop) * contracts * multiplier * FX
- SHORT: max(0, stop - close) * contracts * multiplier * FX

Inputs expected in BACKTEST_OUTPUT_ROOT:
- portfolio_equity.(csv|parquet)  [required]
- positions.(csv|parquet)         [required]
- trades.csv                      [required]
- instrument_map.csv              [optional; maps trade_symbol -> signal_symbol]

Also reads contract specs (multiplier + margin/contract if available) from:
- 01-futures_universe/futures_contracts_full.parquet

And reads signal close + vol (for stop computation) from:
- 04-futures_price_volatility/.../parquet/<SIGNAL_SYMBOL>.parquet
===============================================================================
"""

# ============================================================
# Configuration
# ============================================================

CONFIG = {
    # Backtest output folder (engine outputs)
    "backtest_output_root": Path("./05-futures_backtest_position_sizing_mapped"),

    # Contract specs (multiplier + margin columns)
    "contracts_file": Path("./01-futures_universe/futures_contracts_full.parquet"),
    "margin_column_candidates": [
        "initial_margin",
        "init_margin",
        "margin",
        "initial_margin_usd",
        "margin_usd",
    ],

    # Signal data folder (to pull close + ewma32_std for stop calculation)
    "signal_data_dir": Path("./04-futures_price_volatility/norgate_continuous/volatility_measures/parquet"),
    "signal_close_column": "close",
    "signal_vol_column": "ewma32_std",   # DAILY vol

    # FX (single scalar for now)
    "fx": 1.0,

    # Stop-loss model: stop = entry_px * (1 +/- k * sigma_daily)
    "stop_loss_k": 3.0,

    # Lot closing rule
    "close_rule": "FIFO",  # "FIFO" or "LIFO"

    # Output
    "report_dir": Path("./06-reporting"),
    "pdf_name": "daily_statements.pdf",

    # Render options
    "max_tx_rows_per_bucket": 18,
    "max_lot_rows": 22,

    # If contracts file lacks margin, override per trade symbol here:
    "margin_overrides": {
        # "MES": 1500.0
    },
}

# ============================================================
# Utilities
# ============================================================

def fmt_money(x) -> str:
    if x is None or (isinstance(x, float) and not np.isfinite(x)):
        return "N/A"
    return f"{float(x):,.2f}"

def fmt_pct(x) -> str:
    if x is None or (isinstance(x, float) and not np.isfinite(x)):
        return "N/A"
    return f"{100.0*float(x):.2f}%"

def load_df_prefer_parquet(root: Path, stem: str) -> pd.DataFrame:
    pq = root / f"{stem}.parquet"
    csv = root / f"{stem}.csv"
    if pq.exists():
        return pd.read_parquet(pq)
    if csv.exists():
        return pd.read_csv(csv)
    raise FileNotFoundError(f"Missing {stem}.parquet or {stem}.csv in {root}")

def normalize_dates(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors="coerce")
    return df.dropna(subset=[col]).sort_values(col)

def load_instrument_map(backtest_root: Path) -> dict:
    """trade_symbol -> signal_symbol if instrument_map.csv exists; else {}."""
    p = backtest_root / "instrument_map.csv"
    if not p.exists():
        return {}
    m = pd.read_csv(p)
    m.columns = [str(c).strip().lower() for c in m.columns]
    if "trade_symbol" not in m.columns or "signal_symbol" not in m.columns:
        return {}
    out = {}
    for _, r in m.iterrows():
        ts = str(r["trade_symbol"]).strip().upper()
        ss = str(r["signal_symbol"]).strip().upper()
        if ts and ss and ts.lower() != "nan" and ss.lower() != "nan":
            out[ts] = ss
    return out

def load_contract_specs(contracts_file: Path, margin_candidates: list[str], margin_overrides: dict):
    """Return multiplier_map, margin_map keyed by symbol."""
    if not contracts_file.exists():
        raise FileNotFoundError(f"contracts_file not found: {contracts_file}")

    df = pd.read_parquet(contracts_file)
    df.columns = [str(c).strip().lower() for c in df.columns]

    sym_col_candidates = ["symbol", "ticker", "root", "contract", "instrument"]
    sym_col = next((c for c in sym_col_candidates if c in df.columns), None)
    if sym_col is None:
        raise ValueError(f"Could not find symbol column in contracts file. Columns: {list(df.columns)}")

    pv_col_candidates = ["point_value", "pointvalue", "multiplier", "contract_multiplier"]
    pv_col = next((c for c in pv_col_candidates if c in df.columns), None)
    if pv_col is None:
        raise ValueError(f"Could not find point value column in contracts file. Columns: {list(df.columns)}")

    margin_col = next((c for c in margin_candidates if c in df.columns), None)

    mult_map = {}
    margin_map = {}

    for _, r in df.iterrows():
        sym = str(r[sym_col]).strip().upper()
        if not sym or sym.lower() == "nan":
            continue
        pv = pd.to_numeric(r[pv_col], errors="coerce")
        if pd.notna(pv) and float(pv) > 0:
            mult_map[sym] = float(pv)

        if margin_col is not None:
            mg = pd.to_numeric(r[margin_col], errors="coerce")
            if pd.notna(mg) and float(mg) > 0:
                margin_map[sym] = float(mg)

    for k, v in (margin_overrides or {}).items():
        margin_map[str(k).strip().upper()] = float(v)

    return mult_map, margin_map

def load_signal_series(signal_data_dir: Path, signal_symbol: str, date_index: pd.DatetimeIndex,
                       close_col: str, vol_col: str) -> pd.DataFrame:
    """Return df indexed by date with close and daily vol."""
    p = signal_data_dir / f"{signal_symbol}.parquet"
    if not p.exists():
        raise FileNotFoundError(f"Signal data file not found: {p}")

    df = pd.read_parquet(p)
    df.columns = [str(c).strip().lower() for c in df.columns]
    if "date" not in df.columns:
        if isinstance(df.index, pd.DatetimeIndex):
            df = df.reset_index().rename(columns={"index": "date"})
        else:
            raise ValueError(f"Signal file missing date: {p.name}")

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"]).sort_values("date").set_index("date")

    if close_col not in df.columns:
        raise ValueError(f"Signal file missing close '{close_col}': {p.name}")
    if vol_col not in df.columns:
        raise ValueError(f"Signal file missing vol '{vol_col}': {p.name}")

    out = df[[close_col, vol_col]].copy()
    return out.reindex(date_index)

# ============================================================
# Transaction classification (BTO/STO/BTC/STC)
# ============================================================

def split_trade_actions(prev: int, new: int):
    """Return list of (action, qty>0). Handles sign flips."""
    actions = []
    if prev == new:
        return actions
    if prev == 0:
        if new > 0:
            return [("BTO", new)]
        if new < 0:
            return [("STO", abs(new))]
        return []
    if new == 0:
        if prev > 0:
            return [("STC", prev)]
        if prev < 0:
            return [("BTC", abs(prev))]
        return []

    if (prev > 0 and new > 0) or (prev < 0 and new < 0):
        if abs(new) > abs(prev):
            delta = abs(new) - abs(prev)
            return [("BTO", delta)] if new > 0 else [("STO", delta)]
        delta = abs(prev) - abs(new)
        return [("STC", delta)] if prev > 0 else [("BTC", delta)]

    if prev > 0 and new < 0:
        return [("STC", prev), ("STO", abs(new))]
    if prev < 0 and new > 0:
        return [("BTC", abs(prev)), ("BTO", new)]
    return actions

# ============================================================
# Lot accounting
# ============================================================

def compute_stop(entry_px: float, sigma_daily: float, side: str, k: float) -> float:
    if not (np.isfinite(entry_px) and entry_px > 0 and np.isfinite(sigma_daily) and sigma_daily > 0):
        return np.nan
    if side == "LONG":
        return float(entry_px * (1.0 - k * sigma_daily))
    return float(entry_px * (1.0 + k * sigma_daily))

def close_lots(lots: list[dict], qty_to_close: int, close_rule: str):
    """Close qty from lots FIFO/LIFO. Return (remaining_lots, closed_slices)."""
    if qty_to_close <= 0 or not lots:
        return lots, []
    close_rule = close_rule.upper()
    idxs = range(len(lots)) if close_rule == "FIFO" else range(len(lots)-1, -1, -1)

    remaining = [l.copy() for l in lots]
    closed = []

    for i in list(idxs):
        if qty_to_close <= 0:
            break
        lot = remaining[i]
        q = int(lot.get("qty", 0))
        if q <= 0:
            continue
        take = min(q, qty_to_close)
        qty_to_close -= take
        lot["qty"] = q - take
        sl = lot.copy()
        sl["qty_closed"] = int(take)
        closed.append(sl)

    remaining = [l for l in remaining if int(l.get("qty", 0)) > 0]
    return remaining, closed

# ============================================================
# Statement generation
# ============================================================

def build_daily_statements(portfolio: pd.DataFrame,
                           trades: pd.DataFrame,
                           positions: pd.DataFrame,
                           trade_to_signal: dict,
                           mult_map: dict,
                           margin_map: dict) -> list[dict]:
    fx = float(CONFIG["fx"])
    k_stop = float(CONFIG["stop_loss_k"])
    close_rule = str(CONFIG["close_rule"]).upper()

    portfolio = portfolio.copy()
    portfolio.columns = [str(c).strip().lower() for c in portfolio.columns]
    portfolio = normalize_dates(portfolio, "date").reset_index(drop=True)

    trades = trades.copy()
    trades.columns = [str(c).strip().lower() for c in trades.columns]
    trades = normalize_dates(trades, "date").reset_index(drop=True)

    positions = positions.copy()
    date_col = next((c for c in positions.columns if str(c).strip().lower() == "date"), None)
    if date_col != "date":
        positions = positions.rename(columns={date_col: "date"})
    positions = normalize_dates(positions, "date").reset_index(drop=True)

    dates = pd.to_datetime(portfolio["date"]).drop_duplicates().sort_values().to_list()
    date_index = pd.DatetimeIndex(dates)

    trade_syms = [c for c in positions.columns if c != "date"]

    signal_cache = {}
    for ts in trade_syms:
        ss = trade_to_signal.get(str(ts).upper(), str(ts).upper())
        if ss in signal_cache:
            continue
        try:
            signal_cache[ss] = load_signal_series(
                CONFIG["signal_data_dir"], ss, date_index,
                CONFIG["signal_close_column"], CONFIG["signal_vol_column"]
            )
        except Exception:
            signal_cache[ss] = pd.DataFrame(index=date_index, data={
                CONFIG["signal_close_column"]: np.nan,
                CONFIG["signal_vol_column"]: np.nan,
            })

    trades_by_date = {d: g for d, g in trades.groupby("date")}
    lots_state = {ts: {"LONG": [], "SHORT": []} for ts in trade_syms}

    def get_close_sigma(trade_symbol: str, dt: pd.Timestamp):
        ss = trade_to_signal.get(str(trade_symbol).upper(), str(trade_symbol).upper())
        ser = signal_cache.get(ss)
        if ser is None or dt not in ser.index:
            return np.nan, np.nan
        c = ser.at[dt, CONFIG["signal_close_column"]]
        s = ser.at[dt, CONFIG["signal_vol_column"]]
        return (float(c) if pd.notna(c) else np.nan), (float(s) if pd.notna(s) else np.nan)

    statements = []

    for i, dt in enumerate(dates):
        dt = pd.Timestamp(dt)

        prow = portfolio.loc[portfolio["date"] == dt]
        if prow.empty:
            continue
        eq = float(prow["equity"].iloc[0])
        pnl = float(prow["pnl"].iloc[0]) if "pnl" in prow.columns and pd.notna(prow["pnl"].iloc[0]) else np.nan
        comm_day = float(prow["commissions"].iloc[0]) if "commissions" in prow.columns and pd.notna(prow["commissions"].iloc[0]) else 0.0

        tx_rows = []
        realized_closed_pnl = 0.0

        todays_trades = trades_by_date.get(dt, None)
        if todays_trades is not None and not todays_trades.empty:
            for _, tr in todays_trades.iterrows():
                if "trade_symbol" in todays_trades.columns:
                    ts = str(tr.get("trade_symbol", "")).strip().upper()
                else:
                    ts = str(tr.get("symbol", "")).strip().upper()
                if not ts or ts.lower() == "nan":
                    continue
                if ts not in lots_state:
                    lots_state[ts] = {"LONG": [], "SHORT": []}

                prev_c = int(tr.get("prev_contracts", 0))
                new_c = int(tr.get("new_contracts", 0))
                actions = split_trade_actions(prev_c, new_c)

                close_px, sigma_daily = get_close_sigma(ts, dt)
                mult = float(mult_map.get(ts, np.nan))
                mgn_ct = float(margin_map.get(ts, np.nan))

                for action, qty in actions:
                    qty = int(qty)
                    if qty <= 0:
                        continue

                    if action in ("BTO", "STO"):
                        side = "LONG" if action == "BTO" else "SHORT"
                        entry_px = close_px
                        stop_px = compute_stop(entry_px, sigma_daily, side, k_stop)

                        lot = {
                            "trade_symbol": ts,
                            "side": side,
                            "qty": qty,
                            "entry_date": dt.date().isoformat(),
                            "entry_price": float(entry_px) if np.isfinite(entry_px) else np.nan,
                            "stop_loss_price": float(stop_px) if np.isfinite(stop_px) else np.nan,
                            "multiplier": mult,
                            "margin_per_contract": mgn_ct,
                            "fx": fx,
                        }
                        lots_state[ts][side].append(lot)

                        tx_rows.append({
                            "date": dt.date().isoformat(),
                            "trade_symbol": ts,
                            "action": action,
                            "qty": qty,
                            "exec_price": entry_px,
                            "stop_loss_price": stop_px,
                            "commission": float(tr.get("commission", 0.0)) if pd.notna(tr.get("commission", np.nan)) else 0.0,
                        })

                    elif action in ("BTC", "STC"):
                        side = "SHORT" if action == "BTC" else "LONG"
                        lots = lots_state[ts][side]
                        remaining, closed_slices = close_lots(lots, qty, close_rule)

                        for sl in closed_slices:
                            q_closed = int(sl.get("qty_closed", 0))
                            entry_px = float(sl.get("entry_price", np.nan))
                            if np.isfinite(entry_px) and np.isfinite(close_px) and np.isfinite(mult):
                                if side == "LONG":
                                    rpnl = q_closed * mult * fx * (close_px - entry_px)
                                else:
                                    rpnl = q_closed * mult * fx * (entry_px - close_px)
                            else:
                                rpnl = np.nan
                            realized_closed_pnl += float(rpnl) if np.isfinite(rpnl) else 0.0

                            tx_rows.append({
                                "date": dt.date().isoformat(),
                                "trade_symbol": ts,
                                "action": action,
                                "qty": q_closed,
                                "exec_price": close_px,
                                "closed_entry_date": sl.get("entry_date"),
                                "closed_entry_price": entry_px,
                                "realized_pnl": float(rpnl) if np.isfinite(rpnl) else np.nan,
                                "commission": float(tr.get("commission", 0.0)) if pd.notna(tr.get("commission", np.nan)) else 0.0,
                            })

                        lots_state[ts][side] = remaining

        open_lots = []
        margin_used = 0.0
        total_notional_gross = 0.0
        total_risk_to_stop = 0.0

        for ts in trade_syms:
            close_px, _ = get_close_sigma(ts, dt)
            mult = float(mult_map.get(ts, np.nan))
            mgn_ct = float(margin_map.get(ts, np.nan))

            for side in ("LONG", "SHORT"):
                for lot in lots_state.get(ts, {}).get(side, []):
                    q = int(lot.get("qty", 0))
                    if q <= 0:
                        continue

                    entry_px = float(lot.get("entry_price", np.nan))
                    stop_px = float(lot.get("stop_loss_price", np.nan))
                    entry_date = lot.get("entry_date", "")

                    notional = abs(q) * mult * fx * close_px if (np.isfinite(close_px) and np.isfinite(mult)) else np.nan
                    mgn_used = abs(q) * mgn_ct if np.isfinite(mgn_ct) else np.nan

                    # Unrealized PnL (mark-to-market)
                    if np.isfinite(close_px) and np.isfinite(entry_px) and np.isfinite(mult):
                        upnl = q * mult * fx * (close_px - entry_px) if side == "LONG" else q * mult * fx * (entry_px - close_px)
                    else:
                        upnl = np.nan

                    # Risk to stop (loss if stopped from current close -> stop level)
                    if np.isfinite(close_px) and np.isfinite(stop_px) and np.isfinite(mult):
                        if side == "LONG":
                            risk_to_stop = max(0.0, (close_px - stop_px)) * abs(q) * mult * fx
                        else:
                            risk_to_stop = max(0.0, (stop_px - close_px)) * abs(q) * mult * fx
                    else:
                        risk_to_stop = np.nan

                    open_lots.append({
                        "trade_symbol": ts,
                        "side": side,
                        "qty": q,
                        "entry_date": entry_date,
                        "entry_price": entry_px,
                        "stop_loss_price": stop_px,
                        "close": close_px,
                        "risk_to_stop": risk_to_stop,
                        "risk_to_stop_pct_equity": (risk_to_stop / eq) if (np.isfinite(risk_to_stop) and eq and eq > 0) else np.nan,
                        "multiplier": mult,
                        "notional": notional,
                        "margin_per_contract": mgn_ct,
                        "margin_used": mgn_used,
                        "unrealized_pnl": upnl,
                    })

                    if np.isfinite(mgn_used):
                        margin_used += float(mgn_used)
                    if np.isfinite(notional):
                        total_notional_gross += float(abs(notional))
                    if np.isfinite(risk_to_stop):
                        total_risk_to_stop += float(risk_to_stop)

        margin_util_pct = (margin_used / eq) if (eq and eq > 0) else np.nan
        notional_leverage = (total_notional_gross / eq) if (eq and eq > 0) else np.nan
        risk_to_stop_pct_equity = (total_risk_to_stop / eq) if (eq and eq > 0) else np.nan

        prev_eq = np.nan
        if i > 0:
            prev_row = portfolio.loc[portfolio["date"] == pd.Timestamp(dates[i-1])]
            if not prev_row.empty:
                prev_eq = float(prev_row["equity"].iloc[0])
        closed_trade_return = (realized_closed_pnl / prev_eq) if (np.isfinite(prev_eq) and prev_eq != 0) else np.nan

        tx_df = pd.DataFrame(tx_rows) if tx_rows else pd.DataFrame(columns=["date","trade_symbol","action","qty","exec_price"])
        tx_buckets = {k: [] for k in ["BTO", "STO", "BTC", "STC"]}
        if not tx_df.empty:
            for k in tx_buckets:
                tx_buckets[k] = tx_df.loc[tx_df["action"] == k].to_dict("records")

        open_lots = sorted(open_lots, key=lambda r: (r["trade_symbol"], r["side"], r["entry_date"], r["entry_price"]))

        statements.append({
            "date": dt.date().isoformat(),
            "equity": eq,
            "pnl": pnl,
            "commissions": comm_day,
            "margin_used": margin_used,
            "margin_util_pct": margin_util_pct,
            "total_notional_gross": total_notional_gross,
            "notional_leverage": notional_leverage,
            "total_risk_to_stop": total_risk_to_stop,
            "risk_to_stop_pct_equity": risk_to_stop_pct_equity,
            "tx_buckets": tx_buckets,
            "realized_closed_pnl": realized_closed_pnl,
            "closed_trade_return": closed_trade_return,
            "open_lots": open_lots,
        })

    return statements

# ============================================================
# PDF Rendering
# ============================================================

def draw_table(c, left, y, col_specs, rows, max_rows, title=None):
    if title:
        c.setFont("Helvetica-Bold", 10)
        c.drawString(left, y, title)
        y -= 0.16 * inch

    c.setFont("Helvetica-Bold", 8)
    x = left
    for header, _key, w in col_specs:
        c.drawString(x, y, header)
        x += w * inch
    y -= 0.14 * inch
    c.setFont("Helvetica", 8)

    for r in rows[:max_rows]:
        x = left
        for _header, key, w in col_specs:
            val = r.get(key, "")
            if isinstance(val, float):
                if key in ("margin_util_pct", "closed_trade_return", "risk_to_stop_pct_equity"):
                    s = fmt_pct(val)
                elif key in ("qty",):
                    s = str(int(val))
                elif key in ("exec_price","entry_price","stop_loss_price","notional","margin_used","unrealized_pnl","close",
                             "multiplier","margin_per_contract","realized_pnl","closed_entry_price","risk_to_stop"):
                    s = fmt_money(val)
                else:
                    s = f"{val:.4f}"
            else:
                s = str(val)
            c.drawString(x, y, s[:22])
            x += w * inch
        y -= 0.14 * inch
        if y < 0.9 * inch:
            break

    if len(rows) > max_rows:
        c.setFont("Helvetica-Oblique", 8)
        c.drawString(left, y, f"... ({len(rows) - max_rows} more)")
        y -= 0.16 * inch
        c.setFont("Helvetica", 8)

    return y

def write_daily_statement_pdf(statements: list[dict], out_pdf: Path):
    c = canvas.Canvas(str(out_pdf), pagesize=letter)
    width, height = letter
    left = 0.65 * inch
    top = height - 0.65 * inch

    for st in statements:
        y = top

        c.setFont("Helvetica-Bold", 14)
        c.drawString(left, y, f"Daily Statement - {st['date']}")
        y -= 0.30 * inch

        c.setFont("Helvetica", 10)
        c.drawString(left, y, f"Account Equity: ${fmt_money(st['equity'])}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Margin Utilization ($): ${fmt_money(st['margin_used'])}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Margin Utilization (%): {fmt_pct(st['margin_util_pct'])}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Total Notional Exposure (Gross): ${fmt_money(st.get('total_notional_gross', 0.0))}")
        y -= 0.18 * inch
        nl = st.get("notional_leverage", np.nan)
        c.drawString(left, y, f"Notional / Equity: {nl:.2f}x" if np.isfinite(nl) else "Notional / Equity: N/A")
        y -= 0.18 * inch
        c.drawString(left, y, f"Total Risk to Stops ($): ${fmt_money(st.get('total_risk_to_stop', 0.0))}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Risk to Stops (% Equity): {fmt_pct(st.get('risk_to_stop_pct_equity', np.nan))}")
        y -= 0.22 * inch

        c.drawString(left, y, f"Realized P&L (Closings): ${fmt_money(st.get('realized_closed_pnl', 0.0))}")
        y -= 0.18 * inch
        c.drawString(left, y, f"Return on Closed Trades (vs prior equity): {fmt_pct(st.get('closed_trade_return', np.nan))}")
        y -= 0.28 * inch

        c.setFont("Helvetica-Bold", 11)
        c.drawString(left, y, "New Transactions")
        y -= 0.22 * inch

        tx_cols = [
            ("Sym", "trade_symbol", 0.7),
            ("Act", "action", 0.45),
            ("Qty", "qty", 0.45),
            ("Px", "exec_price", 0.8),
            ("EntryDt", "closed_entry_date", 0.8),
            ("EntryPx", "closed_entry_price", 0.8),
            ("R-PnL", "realized_pnl", 0.8),
        ]
        for bucket in ["BTO", "STO", "BTC", "STC"]:
            rows = st["tx_buckets"].get(bucket, [])
            if rows:
                y = draw_table(c, left, y, tx_cols, rows, CONFIG["max_tx_rows_per_bucket"], title=bucket)
                y -= 0.10 * inch

        if y < 2.2 * inch:
            c.showPage()
            y = top

        c.setFont("Helvetica-Bold", 11)
        c.drawString(left, y, "Existing Portfolio (Lots)")
        y -= 0.22 * inch

        lot_cols = [
            ("Sym", "trade_symbol", 0.55),
            ("Side", "side", 0.4),
            ("Qty", "qty", 0.32),
            ("EntryDt", "entry_date", 0.70),
            ("EntryPx", "entry_price", 0.65),
            ("Stop", "stop_loss_price", 0.65),
            ("Close", "close", 0.55),
            ("Risk$", "risk_to_stop", 0.65),
            ("Risk%", "risk_to_stop_pct_equity", 0.55),
            ("Mult", "multiplier", 0.55),
            ("Notional", "notional", 0.75),
            ("Mgn/ct", "margin_per_contract", 0.55),
            ("Mgn$", "margin_used", 0.55),
            ("uPnL", "unrealized_pnl", 0.55),
        ]

        open_lots = st.get("open_lots", [])
        y = draw_table(c, left, y, lot_cols, open_lots, CONFIG["max_lot_rows"], title=None)

        c.showPage()

    c.save()

# ============================================================
# Main
# ============================================================

def main():
    root = Path(CONFIG["backtest_output_root"])
    if not root.exists():
        raise FileNotFoundError(f"Backtest output root not found: {root}")

    portfolio = load_df_prefer_parquet(root, "portfolio_equity")
    positions = load_df_prefer_parquet(root, "positions")
    trades_path = root / "trades.csv"
    if not trades_path.exists():
        raise FileNotFoundError(f"trades.csv not found in {root}")
    trades = pd.read_csv(trades_path)

    trade_to_signal = load_instrument_map(root)
    mult_map, margin_map = load_contract_specs(
        CONFIG["contracts_file"],
        CONFIG["margin_column_candidates"],
        CONFIG.get("margin_overrides", {}),
    )

    statements = build_daily_statements(
        portfolio=portfolio,
        trades=trades,
        positions=positions,
        trade_to_signal=trade_to_signal,
        mult_map=mult_map,
        margin_map=margin_map,
    )

    out_dir = Path(CONFIG["report_dir"])
    out_dir.mkdir(parents=True, exist_ok=True)
    out_pdf = out_dir / CONFIG["pdf_name"]

    write_daily_statement_pdf(statements, out_pdf)

    print(f"Daily statements PDF written to: {out_pdf.resolve()}")
    print(f"Pages: {len(statements)}")

if __name__ == "__main__":
    main()


Daily statements PDF written to: C:\TWS API\source\pythonclient\TradingIdeas\Futures\06-reporting\daily_statements.pdf
Pages: 432
